## Парсер

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

In [2]:
import time
import numpy as np
from tqdm import tqdm

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [20]:
chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--start-fullscreen")
chrome_options.add_argument("--no-proxy-server")
chrome_options.add_argument("--proxy-server='direct://'")
chrome_options.add_argument("--proxy-bypass-list=*")

In [10]:
gen_info = [] ### Этот код для того того, чтобы получить ссылки на страницу каждого университета на сайте
url = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2023?qs_qp=topnav'

### И еще важно, чтобы код проработал нормально, окно браузера должно быть полностью открыто. 


with webdriver.Chrome() as browser:
    browser.get(url) ### это главная страница, на которой находятся все ссылки
    browser.maximize_window()
    time.sleep(1)
    
    ### На странице представлено более 1000 университетов. При этом на данный момент в html коде видна инфа только по 10 университетам.
    ### Чтобы это исправить, надо увеличить диапозов видимых университетов до 100. Так будет быстрее
    
    ### В коде ниже мы находим необходимый dropdown, после чего прокручиваем для него страницу. Scrollby как костыль, потому что 
    ### на моем мониторе кнопку после скроллинга перекрывало меню, из-за чего я чутка прокручиваю страницу на вверх
    ### Кстати, к проверяющему: если активируйте код, то, возможно, скроллинга в -100 будет не хватать (или будет слишком много)
    ### Из-за чего кнопка будет не видная и на нее не нажмешь. Нужно будет этот момент проследить.
    change_name = browser.find_elements(By.XPATH, "//div[@class = 'ui fluid  selection dropdown _semantic_select newzindex sort_by_dropdown perpage_dropdown_js inline-block minwidth84 pull-left margin-right-px']")[-1]
    browser.execute_script("return arguments[0].scrollIntoView(true);", change_name)
    browser.execute_script("window.scrollBy(0,-100)")
    change_name.click()
    time.sleep(1)
    
    ### Тут жмем на кнопку и получаем дисплей на 100 университетов
    
    button_100 = browser.find_element(By.XPATH, "//div[@class = 'menu transition visible']/div[4]")
    button_100.click()
    
    time.sleep(10)
    
    ### тут сохраняем кнопочку, на которую нажимаем, когда хотим получить следующие 100 университетов
    button = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@class = 'page-link next']")))
    print(button.location)
    
    ### Сделал while, так как в рейтинге у нас 1420 университетов
    while len(gen_info) < 1420:
        
        
        names_links = browser.find_elements(By.CLASS_NAME, 'uni-link') ### Тут узел, создержаштй ссылки
        
        
        names = [i.text for i in names_links] ### Тут имена вузов
        
        rank = [i.text for i in browser.find_elements(By.XPATH, "//div[@class = '_univ-rank hide-this-in-mobile-indi ']")] # позиция
        ### вуза в рейтинге
        
        links = [i.get_attribute('href') for i in names_links] ### Тут уже непосредственно сами ссылки
        
        locations = [i.text for i in browser.find_elements(By.CLASS_NAME, 'location')] ### тут местоположение вуза
        
        ### Здесь итоговые баллые в рейтинге
        points = [i.text for i in browser.find_elements(By.XPATH, "//span[@class = 'overall-score-span hide-this-in-mobile-indi']")] 
        
        first_info = list(zip(rank,names,links,locations,points)) ### тут собираем всю инфу в один tuple
        
        gen_info.extend(first_info) ### и добавляем в общий пул с другими университетами
        
        print(first_info)
        
    
        
        
        ### Тут на всякий пожарный еще рас сохраняю кнопку для нахождения вуза. Когда дойдет до 1420, то кнопка не будет clickable.
        ### Честно, когда комментарии писал - задумался, зачем тут этот трай, но до конца не пойму. Видимо, у меня была какая-то
        ### глубока мысль, из-за которой я решил это использовать
        
        try: 
            button = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@class = 'page-link next']")))
        except Exception:
            print('Спарсили, мэл')
            break

        browser.execute_script("return arguments[0].scrollIntoView(true);", button)
        browser.execute_script("window.scrollBy(0,-200)")
        time.sleep(10)
        
        button.click()
        
    

KeyboardInterrupt: 

In [17]:
data = pd.DataFrame(gen_info, columns=['rank', 'name', 'link', 'location','points'])

In [ ]:
circle_dict = {}
student_stats_dict = {}

did_not_work = {}


### Тут все более прозаично. Много траев, чтобы из-за одной неспарсенной ячейки код не ломался, а если будет ошибка,
### то будем знать где. Кстати, суммарно парсилось долго.

with webdriver.Chrome(options=chrome_options) as browser:
    browser.get(url1)
    browser.maximize_window()
    for i,j in tqdm(list(zip(data['link'].values, data['name'].values))):
        
        ### Заходим на страничку сайта
        try:
            browser.get(i)
        except Exception:
            print('Страница не загрузилась')
            print(j)
            did_not_work.setdefault(j,i)
            continue
         
        ### Ждеем пока прогрузиться локация (эмпирически выяснил, что если локация прогрузилась, то прогрузилась и вся остальная инфа)
        try:
        
            WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'circle')))
            
            ### Тут оценки по 100 шкале за различные показатели
            circles = [tuple(i.text.split('\n')) for i in browser.find_elements(By.CLASS_NAME, 'circle')]
            circle_dict.setdefault(j,circles)
        except Exception:
            print('Затупил на кружочках')
            print(j)
            did_not_work.setdefault(j,i)
            continue
            
        try:   
            ### Тут количество работяг в вузе (то есть преподавателей). Но там опять надо жмыкнуть на кнопку, чтобы найти всю инфу
            stud_staff_but = browser.find_element(By.ID, 'studStaff_Tab')
            browser.execute_script("return arguments[0].scrollIntoView(true);", stud_staff_but)
            browser.execute_script("window.scrollBy(0,-400)")
            stud_staff_but.click()
        except Exception:
            print('Затупил с кнопкой')
            print(j)
            did_not_work.setdefault(j,i)
            continue
        
        try:
            ### Тут уже получаем инфу по количеству студентов.
            WebDriverWait(browser,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'studstaff-subsection-title')))
            WebDriverWait(browser,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'studstaff-subsection-count')))
            stats_title = [i.text for i in browser.find_elements(By.CLASS_NAME, 'studstaff-subsection-title')]
            stats_count = [i.text for i in browser.find_elements(By.CLASS_NAME, 'studstaff-subsection-count')]
            all_stats = list(zip(stats_title, stats_count))
            student_stats_dict.setdefault(j, all_stats)
        except Exception:
            print('Опять затупил на студентах')
            print(j)
            did_not_work.setdefault(j,i)
            continue
        


### Демонстрация кода

Так как суммарно все грузится долго - покажу на примере 50 вузов, как работает код и как мы потом собирали датафрейм

In [22]:
gen_info = [] ### Этот код для того того, чтобы получить ссылки на страницу каждого университета на сайте
url = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2023?qs_qp=topnav'

### И еще важно, чтобы код проработал нормально, окно браузера должно быть полностью открыто. 


with webdriver.Chrome() as browser:
    browser.get(url) ### это главная страница, на которой находятся все ссылки
    browser.maximize_window()
    time.sleep(1)
    
    ### На странице представлено более 1000 университетов. При этом на данный момент в html коде видна инфа только по 10 университетам.
    ### Чтобы это исправить, надо увеличить диапозов видимых университетов до 100. Так будет быстрее
    
    ### В коде ниже мы находим необходимый dropdown, после чего прокручиваем для него страницу. Scrollby как костыль, потому что 
    ### на моем мониторе кнопку после скроллинга перекрывало меню, из-за чего я чутка прокручиваю страницу на вверх
    ### Кстати, к проверяющему: если активируйте код, то, возможно, скроллинга в -100 будет не хватать (или будет слишком много)
    ### Из-за чего кнопка будет не видная и на нее не нажмешь. Нужно будет этот момент проследить.
    change_name = browser.find_elements(By.XPATH, "//div[@class = 'ui fluid  selection dropdown _semantic_select newzindex sort_by_dropdown perpage_dropdown_js inline-block minwidth84 pull-left margin-right-px']")[-1]
    browser.execute_script("return arguments[0].scrollIntoView(true);", change_name)
    browser.execute_script("window.scrollBy(0,-100)")
    change_name.click()
    time.sleep(1)
    
    ### Тут жмем на кнопку и получаем дисплей на 100 университетов
    
    button_100 = browser.find_element(By.XPATH, "//div[@class = 'menu transition visible']/div[4]")
    button_100.click()
    
    time.sleep(10)
    
    ### тут сохраняем кнопочку, на которую нажимаем, когда хотим получить следующие 100 университетов
    button = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@class = 'page-link next']")))
    print(button.location)
    
    ### Сделал while, так как в рейтинге у нас 1420 университетов
    while len(gen_info) < 200:
        
        
        names_links = browser.find_elements(By.CLASS_NAME, 'uni-link') ### Тут узел, создержаштй ссылки
        
        
        names = [i.text for i in names_links] ### Тут имена вузов
        
        rank = [i.text for i in browser.find_elements(By.XPATH, "//div[@class = '_univ-rank hide-this-in-mobile-indi ']")] # позиция
        ### вуза в рейтинге
        
        links = [i.get_attribute('href') for i in names_links] ### Тут уже непосредственно сами ссылки
        
        locations = [i.text for i in browser.find_elements(By.CLASS_NAME, 'location')] ### тут местоположение вуза
        
        ### Здесь итоговые баллые в рейтинге
        points = [i.text for i in browser.find_elements(By.XPATH, "//span[@class = 'overall-score-span hide-this-in-mobile-indi']")] 
        
        first_info = list(zip(rank,names,links,locations,points)) ### тут собираем всю инфу в один tuple
        
        gen_info.extend(first_info) ### и добавляем в общий пул с другими университетами
        
        print(first_info)
        
    
        
        
        ### Тут на всякий пожарный еще рас сохраняю кнопку для нахождения вуза. Когда дойдет до 1420, то кнопка не будет clickable.
        ### Честно, когда комментарии писал - задумался, зачем тут этот трай, но до конца не пойму. Видимо, у меня была какая-то
        ### глубока мысль, из-за которой я решил это использовать
        
        try: 
            button = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@class = 'page-link next']")))
        except Exception:
            print('Спарсили, мэл')
            break

        browser.execute_script("return arguments[0].scrollIntoView(true);", button)
        browser.execute_script("window.scrollBy(0,-200)")
        time.sleep(10)
        
        button.click()
        
    

{'x': 1266, 'y': 10739}
[('1', 'Massachusetts Institute of Technology (MIT)', 'https://www.topuniversities.com/universities/massachusetts-institute-technology-mit', 'Cambridge, United States', '100'), ('2', 'University of Cambridge', 'https://www.topuniversities.com/universities/university-cambridge', 'Cambridge, United Kingdom', '98.8'), ('3', 'Stanford University', 'https://www.topuniversities.com/universities/stanford-university', 'Stanford, United States', '98.5'), ('4', 'University of Oxford', 'https://www.topuniversities.com/universities/university-oxford', 'Oxford, United Kingdom', '98.4'), ('5', 'Harvard University', 'https://www.topuniversities.com/universities/harvard-university', 'Cambridge, United States', '97.6'), ('=6', 'California Institute of Technology (Caltech)', 'https://www.topuniversities.com/universities/california-institute-technology-caltech', 'Pasadena, United States', '97'), ('=6', 'Imperial College London', 'https://www.topuniversities.com/universities/imperi

In [121]:
data = pd.DataFrame(gen_info, columns=['rank', 'name', 'link', 'location','points'])

In [122]:
data.head()

,rank,name,link,location,points
0,1,Massachusetts Institute of Technology (MIT),https://www.topuniversities.com/universities/m...,"Cambridge, United States",100
1,2,University of Cambridge,https://www.topuniversities.com/universities/u...,"Cambridge, United Kingdom",98.8
2,3,Stanford University,https://www.topuniversities.com/universities/s...,"Stanford, United States",98.5
3,4,University of Oxford,https://www.topuniversities.com/universities/u...,"Oxford, United Kingdom",98.4
4,5,Harvard University,https://www.topuniversities.com/universities/h...,"Cambridge, United States",97.6


In [123]:
data_50 = data.iloc[:50]

In [124]:
data_50.shape

(50, 5)

In [33]:
circle_dict = {} ### Для различных показателей
student_stats_dict = {} ### Для количества студентов

did_not_work = {}

### Тут все более прозаично. Много траев, чтобы из-за одной неспарсенной ячейки код не ломался, а если будет ошибка,
### то будем знать где. Кстати, суммарно парсилось долго.

with webdriver.Chrome(options=chrome_options) as browser:
    browser.get(url)
    browser.maximize_window()
    for i,j in tqdm(list(zip(data['link'].values[:50], data['name'].values[:50]))):
        print(j)
        
        ### Заходим на страничку сайта
        try:
            browser.get(i)
        except Exception:
            print('Страница не загрузилась')
            print(j)
            did_not_work.setdefault(j,i)
            continue
         
        ### Ждеем пока прогрузиться локация (эмпирически выяснил, что если локация прогрузилась, то прогрузилась и вся остальная инфа)
        try:
        
            WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'circle')))
            
            ### Тут оценки по 100 шкале за различные показатели
            circles = [tuple(i.text.split('\n')) for i in browser.find_elements(By.CLASS_NAME, 'circle')]
            circle_dict.setdefault(j,circles)
        except Exception:
            print('Затупил на кружочках')
            print(j)
            did_not_work.setdefault(j,i)
            continue
            
        try:   
            ### Тут количество работяг в вузе (то есть преподавателей). Но там опять надо жмыкнуть на кнопку, чтобы найти всю инфу
            stud_staff_but = browser.find_element(By.ID, 'studStaff_Tab')
            browser.execute_script("return arguments[0].scrollIntoView(true);", stud_staff_but)
            browser.execute_script("window.scrollBy(0,-400)")
            stud_staff_but.click()
        except Exception:
            print('Затупил с кнопкой')
            print(j)
            did_not_work.setdefault(j,i)
            continue
        
        try:
            ### Тут уже получаем инфу по количеству студентов.
            WebDriverWait(browser,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'studstaff-subsection-title')))
            WebDriverWait(browser,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'studstaff-subsection-count')))
            stats_title = [i.text for i in browser.find_elements(By.CLASS_NAME, 'studstaff-subsection-title')]
            stats_count = [i.text for i in browser.find_elements(By.CLASS_NAME, 'studstaff-subsection-count')]
            all_stats = list(zip(stats_title, stats_count))
            student_stats_dict.setdefault(j, all_stats)
        except Exception:
            print('Опять затупил на студентах')
            print(j)
            did_not_work.setdefault(j,i)
            continue
        
        print(circle_dict)
        print(student_stats_dict)


  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

Massachusetts Institute of Technology (MIT)


  2%|█▋                                                                                 | 1/50 [00:18<15:24, 18.86s/it]

{'Massachusetts Institute of Technology (MIT)': [('100', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('100', 'Citations per Faculty'), ('100', 'International Faculty Ratio'), ('90', 'International Students Ratio'), ('96.1', 'International Research Network'), ('100', 'Employment Outcomes')]}
{'Massachusetts Institute of Technology (MIT)': [('Total students', '11,035'), ('International students', '3,627'), ('Total faculty staff', '2,919')]}
University of Cambridge


  4%|███▎                                                                               | 2/50 [00:24<08:52, 11.10s/it]

...

{'Massachusetts Institute of Technology (MIT)': [('100', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('100', 'Citations per Faculty'), ('100', 'International Faculty Ratio'), ('90', 'International Students Ratio'), ('96.1', 'International Research Network'), ('100', 'Employment Outcomes')], 'University of Cambridge': [('98.8', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('92.3', 'Citations per Faculty'), ('100', 'International Faculty Ratio'), ('96.3', 'International Students Ratio'), ('99.5', 'International Research Network'), ('100', 'Employment Outcomes')], 'Stanford University': [('98.5', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('99.9', 'Citations per Faculty'), ('99.8', 'International Faculty Ratio'), ('60.3', 'International Students Ratio'), ('96.3', 'International Research Network

 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [11:11<00:20, 20.93s/it]

{'Massachusetts Institute of Technology (MIT)': [('100', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('100', 'Citations per Faculty'), ('100', 'International Faculty Ratio'), ('90', 'International Students Ratio'), ('96.1', 'International Research Network'), ('100', 'Employment Outcomes')], 'University of Cambridge': [('98.8', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('92.3', 'Citations per Faculty'), ('100', 'International Faculty Ratio'), ('96.3', 'International Students Ratio'), ('99.5', 'International Research Network'), ('100', 'Employment Outcomes')], 'Stanford University': [('98.5', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('99.9', 'Citations per Faculty'), ('99.8', 'International Faculty Ratio'), ('60.3', 'International Students Ratio'), ('96.3', 'International Research Network

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [11:29<00:00, 13.79s/it]


{'Massachusetts Institute of Technology (MIT)': [('100', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('100', 'Citations per Faculty'), ('100', 'International Faculty Ratio'), ('90', 'International Students Ratio'), ('96.1', 'International Research Network'), ('100', 'Employment Outcomes')], 'University of Cambridge': [('98.8', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('92.3', 'Citations per Faculty'), ('100', 'International Faculty Ratio'), ('96.3', 'International Students Ratio'), ('99.5', 'International Research Network'), ('100', 'Employment Outcomes')], 'Stanford University': [('98.5', 'Overall'), ('100', 'Academic Reputation'), ('100', 'Employer Reputation'), ('100', 'Faculty Student Ratio'), ('99.9', 'Citations per Faculty'), ('99.8', 'International Faculty Ratio'), ('60.3', 'International Students Ratio'), ('96.3', 'International Research Network

Ниже обработаем показатели, чтобы их было удобно включить в датафрейм

In [125]:
feats_circles = {'name': []}

In [126]:
for i in map(lambda x: x[-1], circle_dict['Massachusetts Institute of Technology (MIT)']):
    feats_circles.setdefault(i, [])

In [127]:
for i in circle_dict:
    
    info = dict(map(lambda x: x[::-1],(circle_dict[i])))
    feats_circles['name'].append(i)
    for j in list(feats_circles)[1:]:
        feats_circles[j].append(info.get(j))
        

In [128]:
feats_circles

{'name': ['Massachusetts Institute of Technology (MIT)',
  'University of Cambridge',
  'Stanford University',
  'University of Oxford',
  'Harvard University',
  'California Institute of Technology (Caltech)',
  'Imperial College London',
  'UCL',
  'ETH Zurich',
  'University of Chicago',
  'National University of Singapore (NUS)',
  'Peking University',
  'University of Pennsylvania',
  'Tsinghua University',
  'The University of Edinburgh',
  'EPFL',
  'Princeton University',
  'Yale University',
  'Nanyang Technological University, Singapore (NTU Singapore)',
  'Cornell University',
  'The University of Hong Kong',
  'Columbia University',
  'The University of Tokyo',
  'Johns Hopkins University',
  'University of Michigan-Ann Arbor',
  'Université PSL',
  'University of California, Berkeley (UCB)',
  'The University of Manchester',
  'Seoul National University',
  'Australian National University (ANU)',
  'McGill University',
  'Northwestern University',
  'The University of Melb

Тут обработаем количество студентов и преподавателей

In [129]:
feats_studs = {'name': []}

In [130]:
for i in map(lambda x: x[0], student_stats_dict['Massachusetts Institute of Technology (MIT)']):
    feats_studs.setdefault(i, [])

In [131]:
for i in student_stats_dict:
    
    info = dict(student_stats_dict[i])
    feats_studs['name'].append(i)
    for j in list(feats_studs)[1:]:
        feats_studs[j].append(info.get(j))

In [132]:
circle_df = pd.DataFrame(feats_circles)

In [133]:
stud_df = pd.DataFrame(feats_studs)

Начинаем объединение в единый датафрейм

In [134]:
prefinal = data.merge(stud_df, left_on = 'name', right_on =  'name')

In [135]:
final = prefinal.merge(circle_df, left_on = 'name', right_on = 'name')

In [136]:
final['rank'] = final['rank'].str.extract('(\d+)', expand=False)

In [137]:
final.head()

,rank,name,link,location,points,Total students,International students,Total faculty staff,Overall,Academic Reputation,Employer Reputation,Faculty Student Ratio,Citations per Faculty,International Faculty Ratio,International Students Ratio,International Research Network,Employment Outcomes
0,1,Massachusetts Institute of Technology (MIT),https://www.topuniversities.com/universities/m...,"Cambridge, United States",100,"11,035","3,627","2,919",100,100,100,100,100,100,90,96.1,100
1,2,University of Cambridge,https://www.topuniversities.com/universities/u...,"Cambridge, United Kingdom",98.8,"20,871","7,865","5,735",98.8,100,100,100,92.3,100,96.3,99.5,100
2,3,Stanford University,https://www.topuniversities.com/universities/s...,"Stanford, United States",98.5,"14,518","3,318","4,694",98.5,100,100,100,99.9,99.8,60.3,96.3,100
3,4,University of Oxford,https://www.topuniversities.com/universities/u...,"Oxford, United Kingdom",98.4,"21,972","9,024","6,650",98.4,100,100,100,90,98.8,98.4,99.9,100
4,5,Harvard University,https://www.topuniversities.com/universities/h...,"Cambridge, United States",97.6,"21,877","5,379","4,480",97.6,100,100,99.4,100,76.9,66.9,100,100


Вот и получили наш красивый датафрейм. У проверяющего может быть резонный вопрос: почему нужно было показывать на примере, если сразу после парсинга данных вы все преобразовали в датафрейм? Объясняю - все было крайне запутано, в юпитерском файле был полнейший беспорядок и вместо того, чтобы разбираться в этом беспорядке, решил показать на маленьком примере.